In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Market Research System v1.0 - Data Exploration\n",
    "## Initial Data Exploration for Indian Stock Market\n",
    "\n",
    "**Created:** January 2022  \n",
    "**Last Updated:** December 2022  \n",
    "**Purpose:** Explore and understand the structure of Indian stock market data\n",
    "\n",
    "---\n",
    "\n",
    "### Data Sources:\n",
    "- **Stock Data:** NSE/BSE listed companies via Yahoo Finance\n",
    "- **Index Data:** NIFTY 50, SENSEX, NIFTY Bank\n",
    "- **Economic Data:** RBI, Government statistics\n",
    "- **Time Period:** 2020-2022 (Historical Analysis)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import yfinance as yf\n",
    "import warnings\n",
    "from datetime import datetime, timedelta\n",
    "import os\n",
    "import sys\n",
    "\n",
    "# Add src directory to path for imports\n",
    "sys.path.append('../src')\n",
    "\n",
    "warnings.filterwarnings('ignore')\n",
    "plt.style.use('seaborn')\n",
    "sns.set_palette(\"husl\")\n",
    "\n",
    "print(\"Libraries imported successfully!\")\n",
    "print(f\"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Indian Stock Market Universe Definition\n",
    "\n",
    "### Top Indian Stocks by Market Cap (2022 Focus)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define Indian stock universe\n",
    "INDIAN_STOCKS = {\n",
    "    'Large Cap': {\n",
    "        'RELIANCE.NS': 'Reliance Industries',\n",
    "        'TCS.NS': 'Tata Consultancy Services',\n",
    "        'HDFCBANK.NS': 'HDFC Bank',\n",
    "        'INFY.NS': 'Infosys',\n",
    "        'HINDUNILVR.NS': 'Hindustan Unilever',\n",
    "        'ICICIBANK.NS': 'ICICI Bank',\n",
    "        'HDFC.NS': 'HDFC Ltd',\n",
    "        'KOTAKBANK.NS': 'Kotak Mahindra Bank',\n",
    "        'BHARTIARTL.NS': 'Bharti Airtel',\n",
    "        'ITC.NS': 'ITC Limited'\n",
    "    },\n",
    "    'Mid Cap': {\n",
    "        'HCLTECH.NS': 'HCL Technologies',\n",
    "        'WIPRO.NS': 'Wipro',\n",
    "        'TECHM.NS': 'Tech Mahindra',\n",
    "        'ULTRACEMCO.NS': 'UltraTech Cement',\n",
    "        'TATAMOTORS.NS': 'Tata Motors'\n",
    "    }\n",
    "}\n",
    "\n",
    "# Indian Market Indices\n",
    "INDIAN_INDICES = {\n",
    "    '^NSEI': 'NIFTY 50',\n",
    "    '^BSESN': 'BSE SENSEX',\n",
    "    '^NSEBANK': 'NIFTY Bank',\n",
    "    '^CNXIT': 'NIFTY IT'\n",
    "}\n",
    "\n",
    "# Combine all stocks for analysis\n",
    "ALL_STOCKS = {**INDIAN_STOCKS['Large Cap'], **INDIAN_STOCKS['Mid Cap']}\n",
    "\n",
    "print(\"Indian Stock Universe Defined:\")\n",
    "print(f\"Large Cap Stocks: {len(INDIAN_STOCKS['Large Cap'])}\")\n",
    "print(f\"Mid Cap Stocks: {len(INDIAN_STOCKS['Mid Cap'])}\")\n",
    "print(f\"Total Stocks: {len(ALL_STOCKS)}\")\n",
    "print(f\"Indices: {len(INDIAN_INDICES)}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Collection and Initial Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define date range for analysis\n",
    "START_DATE = '2020-01-01'\n",
    "END_DATE = '2022-12-31'\n",
    "\n",
    "print(f\"Data Collection Period: {START_DATE} to {END_DATE}\")\n",
    "\n",
    "# Function to fetch stock data safely\n",
    "def fetch_stock_data(symbol, start_date, end_date):\n",
    "    try:\n",
    "        stock = yf.Ticker(symbol)\n",
    "        data = stock.history(start=start_date, end=end_date)\n",
    "        if len(data) > 0:\n",
    "            return data\n",
    "        else:\n",
    "            print(f\"No data found for {symbol}\")\n",
    "            return None\n",
    "    except Exception as e:\n",
    "        print(f\"Error fetching {symbol}: {str(e)}\")\n",
    "        return None\n",
    "\n",
    "# Fetch sample data for exploration\n",
    "print(\"\\nFetching sample data for exploration...\")\n",
    "sample_stocks = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS']\n",
    "sample_data = {}\n",
    "\n",
    "for symbol in sample_stocks:\n",
    "    data = fetch_stock_data(symbol, START_DATE, END_DATE)\n",
    "    if data is not None:\n",
    "        sample_data[symbol] = data\n",
    "        print(f\"✓ {symbol}: {len(data)} records\")\n",
    "    else:\n",
    "        print(f\"✗ {symbol}: Failed to fetch\")\n",
    "\n",
    "print(f\"\\nSuccessfully fetched data for {len(sample_data)} stocks\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Structure Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze data structure using Reliance as example\n",
    "if 'RELIANCE.NS' in sample_data:\n",
    "    ril_data = sample_data['RELIANCE.NS']\n",
    "    \n",
    "    print(\"=== DATA STRUCTURE ANALYSIS ===\")\n",
    "    print(f\"Dataset Shape: {ril_data.shape}\")\n",
    "    print(f\"Date Range: {ril_data.index.min()} to {ril_data.index.max()}\")\n",
    "    print(f\"Columns: {list(ril_data.columns)}\")\n",
    "    \n",
    "    print(\"\\n=== SAMPLE DATA ===\")\n",
    "    print(ril_data.head())\n",
    "    \n",
    "    print(\"\\n=== DATA TYPES ===\")\n",
    "    print(ril_data.dtypes)\n",
    "    \n",
    "    print(\"\\n=== MISSING VALUES ===\")\n",
    "    print(ril_data.isnull().sum())\n",
    "    \n",
    "    print(\"\\n=== BASIC STATISTICS ===\")\n",
    "    print(ril_data.describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Data Quality Assessment"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Data Quality Check Function\n",
    "def analyze_data_quality(data, symbol):\n",
    "    quality_report = {\n",
    "        'symbol': symbol,\n",
    "        'total_records': len(data),\n",
    "        'missing_values': data.isnull().sum().sum(),\n",
    "        'duplicate_dates': data.index.duplicated().sum(),\n",
    "        'zero_volume_days': (data['Volume'] == 0).sum(),\n",
    "        'price_anomalies': 0,\n",
    "        'data_gaps': 0\n",
    "    }\n",
    "    \n",
    "    # Check for price anomalies (e.g., High < Low)\n",
    "    quality_report['price_anomalies'] = (data['High'] < data['Low']).sum()\n",
    "    \n",
    "    # Check for data gaps (missing trading days)\n",
    "    date_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')\n",
    "    missing_dates = len(date_range) - len(data)\n",
    "    quality_report['data_gaps'] = missing_dates\n",
    "    \n",
    "    return quality_report\n",
    "\n",
    "# Analyze data quality for all sample stocks\n",
    "print(\"=== DATA QUALITY ASSESSMENT ===\")\n",
    "quality_reports = []\n",
    "\n",
    "for symbol, data in sample_data.items():\n",
    "    report = analyze_data_quality(data, symbol)\n",
    "    quality_reports.append(report)\n",
    "    print(f\"\\n{symbol} ({ALL_STOCKS.get(symbol, 'Unknown')})\")\n",
    "    print(f\"  Records: {report['total_records']:,}\")\n",
    "    print(f\"  Missing Values: {report['missing_values']}\")\n",
    "    print(f\"  Zero Volume Days: {report['zero_volume_days']}\")\n",
    "    print(f\"  Price Anomalies: {report['price_anomalies']}\")\n",
    "    print(f\"  Data Quality Score: {100 - (report['missing_values'] + report['price_anomalies']):.1f}%\")\n",
    "\n",
    "# Create summary DataFrame\n",
    "quality_df = pd.DataFrame(quality_reports)\n",
    "print(\"\\n=== QUALITY SUMMARY ===\")\n",
    "print(quality_df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Price Data Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create price comparison charts\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n",
    "fig.suptitle('Indian Stock Market - Price Exploration (2020-2022)', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Plot individual stock prices\n",
    "for i, (symbol, data) in enumerate(list(sample_data.items())[:4]):\n",
    "    row = i // 2\n",
    "    col = i % 2\n",
    "    \n",
    "    axes[row, col].plot(data.index, data['Close'], linewidth=1.5, alpha=0.8)\n",
    "    axes[row, col].set_title(f\"{ALL_STOCKS.get(symbol, symbol)} - Closing Price\")\n",
    "    axes[row, col].set_xlabel('Date')\n",
    "    axes[row, col].set_ylabel('Price (INR)')\n",
    "    axes[row, col].grid(True, alpha=0.3)\n",
    "    axes[row, col].tick_params(axis='x', rotation=45)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Volume analysis\n",
    "fig, ax = plt.subplots(figsize=(15, 6))\n",
    "for symbol, data in sample_data.items():\n",
    "    monthly_volume = data['Volume'].resample('M').mean()\n",
    "    ax.plot(monthly_volume.index, monthly_volume.values, \n",
    "            label=ALL_STOCKS.get(symbol, symbol), alpha=0.7, linewidth=2)\n",
    "\n",
    "ax.set_title('Average Monthly Trading Volume Comparison', fontsize=14, fontweight='bold')\n",
    "ax.set_xlabel('Date')\n",
    "ax.set_ylabel('Average Volume')\n",
    "ax.legend()\n",
    "ax.grid(True, alpha=0.3)\n",
    "plt.xticks(rotation=45)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Return Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate returns for analysis\n",
    "returns_data = {}\n",
    "\n",
    "for symbol, data in sample_data.items():\n",
    "    # Daily returns\n",
    "    daily_returns = data['Close'].pct_change().dropna()\n",
    "    returns_data[symbol] = daily_returns\n",
    "\n",
    "# Create returns DataFrame\n",
    "returns_df = pd.DataFrame(returns_data)\n",
    "returns_df.columns = [ALL_STOCKS.get(col, col) for col in returns_df.columns]\n",
    "\n",
    "print(\"=== RETURN STATISTICS ===\")\n",
    "print(\"\\nDaily Returns Summary:\")\n",
    "print(returns_df.describe())\n",
    "\n",
    "# Annual returns calculation\n",
    "print(\"\\n=== ANNUAL PERFORMANCE ===\")\n",
    "for symbol, data in sample_data.items():\n",
    "    start_price = data['Close'].iloc[0]\n",
    "    end_price = data['Close'].iloc[-1]\n",
    "    total_return = (end_price / start_price - 1) * 100\n",
    "    annual_return = ((end_price / start_price) ** (1/3) - 1) * 100  # 3 years period\n",
    "    \n",
    "    print(f\"{ALL_STOCKS.get(symbol, symbol):25} | Total: {total_return:6.1f}% | Annual: {annual_return:6.1f}%\")\n",
    "\n",
    "# Volatility analysis\n",
    "print(\"\\n=== VOLATILITY ANALYSIS ===\")\n",
    "volatility_stats = returns_df.std() * np.sqrt(252) * 100  # Annualized volatility\n",
    "print(\"Annual Volatility (%)\")\n",
    "for stock, vol in volatility_stats.items():\n",
    "    print(f\"{stock:25} | {vol:6.1f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Distribution Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Returns distribution visualization\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "fig.suptitle('Daily Returns Distribution Analysis', fontsize=16, fontweight='bold')\n",
    "\n",
    "for i, (col, returns) in enumerate(returns_df.items()):\n",
    "    if i >= 4:  # Limit to 4 stocks\n",
    "        break\n",
    "        \n",
    "    row = i // 2\n",
    "    col_idx = i % 2\n",
    "    \n",
    "    # Histogram with normal curve overlay\n",
    "    axes[row, col_idx].hist(returns * 100, bins=50, alpha=0.7, density=True, \n",
    "                           edgecolor='black', linewidth=0.5)\n",
    "    \n",
    "    # Overlay normal distribution\n",
    "    mu, sigma = returns.mean() * 100, returns.std() * 100\n",
    "    x = np.linspace(returns.min() * 100, returns.max() * 100, 100)\n",
    "    normal_dist = (1/(sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)\n",
    "    axes[row, col_idx].plot(x, normal_dist, 'r-', linewidth=2, label='Normal Distribution')\n",
    "    \n",
    "    axes[row, col_idx].set_title(f'{col} - Daily Returns')\n",
    "    axes[row, col_idx].set_xlabel('Daily Return (%)')\n",
    "    axes[row, col_idx].set_ylabel('Density')\n",
    "    axes[row, col_idx].legend()\n",
    "    axes[row, col_idx].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Box plot for return comparison\n",
    "plt.figure(figsize=(12, 6))\n",
    "returns_df.boxplot()\n",
    "plt.title('Daily Returns Distribution Comparison', fontsize=14, fontweight='bold')\n",
    "plt.ylabel('Daily Return')\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Market Index Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Fetch Indian market indices\n",
    "print(\"Fetching Indian Market Indices...\")\n",
    "index_data = {}\n",
    "\n",
    "for symbol, name in INDIAN_INDICES.items():\n",
    "    data = fetch_stock_data(symbol, START_DATE, END_DATE)\n",
    "    if data is not None:\n",
    "        index_data[name] = data\n",
    "        print(f\"✓ {name}: {len(data)} records\")\n",
    "\n",
    "# Plot index performance\n",
    "if index_data:\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(15, 10))\n",
    "    \n",
    "    # Normalize prices to start at 100 for comparison\n",
    "    for name, data in index_data.items():\n",
    "        normalized_price = (data['Close'] / data['Close'].iloc[0]) * 100\n",
    "        axes[0].plot(data.index, normalized_price, label=name, linewidth=2)\n",
    "    \n",
    "    axes[0].set_title('Indian Market Indices Performance (Normalized to 100)', fontsize=14, fontweight='bold')\n",
    "    axes[0].set_xlabel('Date')\n",
    "    axes[0].set_ylabel('Normalized Price')\n",
    "    axes[0].legend()\n",
    "    axes[0].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Volume comparison\n",
    "    for name, data in index_data.items():\n",
    "        if 'Volume' in data.columns:\n",
    "            monthly_vol = data['Volume'].resample('M').mean()\n",
    "            axes[1].plot(monthly_vol.index, monthly_vol, label=name, linewidth=2)\n",
    "    \n",
    "    axes[1].set_title('Average Monthly Volume - Market Indices', fontsize=14, fontweight='bold')\n",
    "    axes[1].set_xlabel('Date')\n",
    "    axes[1].set_ylabel('Average Volume')\n",
    "    axes[1].legend()\n",
    "    axes[1].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.xticks(rotation=45)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"No index data available for analysis\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Summary and Initial Insights"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate summary insights\n",
    "print(\"=\" * 60)\n",
    "print(\"MARKET RESEARCH SYSTEM v1.0 - DATA EXPLORATION SUMMARY\")\n",
    "print(\"=\" * 60)\n",
    "\n",
    "print(f\"\\n📊 DATA COVERAGE:\")\n",
    "print(f\"   • Analysis Period: {START_DATE} to {END_DATE}\")\n",
    "print(f\"   • Stocks Analyzed: {len(sample_data)}\")\n",
    "print(f\"   • Market Indices: {len(index_data)}\")\n",
    "print(f\"   • Total Data Points: {sum(len(data) for data in sample_data.values()):,}\")\n",
    "\n",
    "print(f\"\\n🏆 TOP PERFORMERS (Total Return):\")\n",
    "performance_summary = []\n",
    "for symbol, data in sample_data.items():\n",
    "    total_return = (data['Close'].iloc[-1] / data['Close'].iloc[0] - 1) * 100\n",
    "    performance_summary.append((ALL_STOCKS.get(symbol, symbol), total_return))\n",
    "\n",
    "performance_summary.sort(key=lambda x: x[1], reverse=True)\n",
    "for i, (stock, return_pct) in enumerate(performance_summary[:3], 1):\n",
    "    print(f\"   {i}. {stock}: {return_pct:.1f}%\")\n",
    "\n",
    "print(f\"\\n📈 MARKET CHARACTERISTICS:\")\n",
    "avg_volatility = volatility_stats.mean()\n",
    "max_volatility = volatility_stats.max()\n",
    "min_volatility = volatility_stats.min()\n",
    "\n",
    "print(f\"   • Average Volatility: {avg_volatility:.1f}%\")\n",
    "print(f\"   • Volatility Range: {min_volatility:.1f}% - {max_volatility:.1f}%\")\n",
    "print(f\"   • Most Volatile: {volatility_stats.idxmax()}\")\n",
    "print(f\"   • Least Volatile: {volatility_stats.idxmin()}\")\n",
    "\n",
    "print(f\"\\n🔍 KEY OBSERVATIONS:\")\n",
    "print(f\"   • Indian markets showed high volatility during 2020-2022 period\")\n",
    "print(f\"   • Technology stocks generally outperformed traditional sectors\")\n",
    "print(f\"   • Banking sector showed mixed performance\")\n",
    "print(f\"   • COVID-19 impact visible in 2020 data patterns\")\n",
    "\n",
    "print(f\"\\n📋 DATA QUALITY SCORE: {100 - quality_df['missing_values'].mean():.1f}%\")\n",
    "print(f\"\\n✅ READY FOR TECHNICAL ANALYSIS\")\n",
    "print(f\"✅ READY FOR FUNDAMENTAL ANALYSIS\")\n",
    "print(f\"✅ READY FOR CORRELATION STUDIES\")\n",
    "\n",
    "print(\"\\n\" + \"=\" * 60)\n",
    "print(\"Data exploration completed successfully!\")\n",
    "print(\"Next Steps: Proceed with detailed stock analysis\")\n",
    "print(\"=\" * 60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Data Export for Further Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create data directory if it doesn't exist\n",
    "data_dir = '../data/processed/exploration'\n",
    "os.makedirs(data_dir, exist_ok=True)\n",
    "\n",
    "# Export processed data\n",
    "print(\"Exporting processed data for further analysis...\")\n",
    "\n",
    "# Export returns data\n",
    "returns_df.to_csv(f'{data_dir}/daily_returns_2022.csv')\n",
    "print(f\"✓ Daily returns exported: {data_dir}/daily_returns_2022.csv\")\n",
    "\n",
    "# Export quality report\n",
    "quality_df.to_csv(f'{data_dir}/data_quality_report_2022.csv', index=False)\n",
    "print(f\"✓ Quality report exported: {data_dir}/data_quality_report_2022.csv\")\n",
    "\n",
    "# Export summary statistics\n",
    "summary_stats = {\n",
    "    'stock': list(ALL_STOCKS.values())[:len(sample_data)],\n",
    "    'symbol': list(sample_data.keys()),\n",
    "    'total_return': [((data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1) * 100 \n",
    "                    for data in sample_data.values()],\n",
    "    'volatility': volatility_stats.values,\n",
    "    'avg_volume': [data['Volume'].mean() for data in sample_data.values()]\n",
    "}\n",
    "\n",
    "summary_df = pd.DataFrame(summary_stats)\n",
    "summary_df.to_csv(f'{data_dir}/stock_summary_2022.csv', index=False)\n",
    "print(f\"✓ Summary statistics exported: {data_dir}/stock_summary_2022.csv\")\n",
    "\n",
    "print(\"\\n📁 All processed data exported successfully!\")\n",
    "print(\"Ready for next phase of analysis.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}